In [7]:
import torch
import torch.nn as nn
from Networks.NetworkModels import SequentialLipltLipschitz

In [24]:
# Example network architecture

net1 = nn.Sequential(nn.Conv2d(3, 4, 7),
                     nn.Flatten(),
                     nn.Linear(2704, 10),
                     nn.ReLU(),
                     nn.Linear(10, 2),
                     nn.Tanh(),
                     nn.Linear(2, 20))

net1.load_state_dict(torch.load("Path/to/your/model.pth"))

# Requirements:
 - The network must be a sequential network
 - The network must have a flatten layer after all the convolutional layers
 - The network must only have Linear, Conv2d, and elementwise activations slope bounded between 0 and 1 like ReLU, Tanh, etc.
Note that our code does not check for these requirements and we have not checked if any errors will be thrown if these requirements are not met.

Furthermore, you need to provide the shape of the input to the network, without the batch dimension. That is, if you for example use CIFAR-10, which has images with shape (3, 32, 32), then the sampleInputShape will be (3, 32, 32).
This is needed to initialize the eigenvectors required for Lipschitz calculation. 

Next, since initial "eigenvectors" are random, it is required to run the power iteration algorithm for a reasonable number of iterations to ensure convergence. Here we set it to 10 iterations, but you can change it to whatever you want. Or instead, you can set it to 1 and call the lipschitz calculation function multiple times until the Lipschitz constant converges.

Finally, you need to specify whether you want to calculate the Lipschitz constant per model, per class, or pairwise. Note that only one of these can be calculated at a time by our model.

In [25]:
sampleInputShape = (3, 32, 32)
numberOfPowerIterations = 10
net2 = SequentialLipltLipschitz(list(net1), sampleInputShape, perClassLipschitz=False, numberOfPowerIterations=numberOfPowerIterations, pairwiseLipschitz=True)
net2.cuda()

SequentialLipltLipschitz(
  (linear): Sequential(
    (0): Conv2d(3, 4, kernel_size=(7, 7), stride=(1, 1))
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=2704, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=2, bias=True)
    (5): Tanh()
    (6): Linear(in_features=2, out_features=20, bias=True)
  )
)

In [27]:
net2.calculateNetworkLipschitz(0)

tensor([[0.0000, 0.0640, 0.2161, 0.2609, 0.2430, 0.1005, 0.2007, 0.2664, 0.3343,
         0.2564, 0.3028, 0.2593, 0.3983, 0.2822, 0.2575, 0.1886, 0.2494, 0.4046,
         0.1910, 0.0651],
        [0.0640, 0.0000, 0.2370, 0.2555, 0.1813, 0.0409, 0.1484, 0.2221, 0.3455,
         0.2082, 0.2777, 0.1992, 0.3668, 0.2965, 0.2141, 0.1291, 0.2445, 0.3798,
         0.2107, 0.0699],
        [0.2161, 0.2370, 0.0000, 0.1052, 0.3192, 0.2352, 0.2440, 0.2451, 0.1220,
         0.2556, 0.1948, 0.3188, 0.2880, 0.0689, 0.2365, 0.2697, 0.0981, 0.2701,
         0.0263, 0.2799],
        [0.2609, 0.2555, 0.1052, 0.0000, 0.2705, 0.2366, 0.1974, 0.1670, 0.1124,
         0.1845, 0.0920, 0.2626, 0.1827, 0.0900, 0.1605, 0.2341, 0.0116, 0.1670,
         0.1071, 0.3157],
        [0.2430, 0.1813, 0.3192, 0.2705, 0.0000, 0.1425, 0.0756, 0.1247, 0.3826,
         0.1005, 0.2287, 0.0244, 0.2767, 0.3504, 0.1249, 0.0570, 0.2646, 0.3092,
         0.2975, 0.2391],
        [0.1005, 0.0409, 0.2352, 0.2366, 0.1425, 0.0000, 0.1